<a href="https://colab.research.google.com/github/Waxpple/fakenews/blob/master/Finale_Colab_File_Deep_Faking_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

REF:
# link:"""DeepFaking the News with NLP and Transformer Models fron Alan Geitger
#Original file is located at
#    https://colab.research.google.com/drive/#1VI3oBIOQYsym2x5oOux7DTNhpdR0r4uw


### Step 1: Configuration

In [3]:
!pip install python-wordpress-xmlrpc

  Created wheel for python-wordpress-xmlrpc: filename=python_wordpress_xmlrpc-2.3-cp36-none-any.whl size=16363 sha256=1263cc5546482206334dc1d380905c35ffeaa67ce881b4bb4079708e057de0a1
  Stored in directory: /root/.cache/pip/wheels/4e/6a/e1/765a3b784c2a77f9c8bdaacc4a8f87ca6d54767c5c08ba779e
Successfully built python-wordpress-xmlrpc


In [0]:
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.posts import NewPost

#authenticate
wp_url = "####"
wp_username = "#####"
wp_password = "@#####"

wp = Client(wp_url, wp_username, wp_password)

In [0]:
# Fake person who will be slandered/libeled in the fake articles
NAME_TO_SLANDER = "Peter Godamn"
IMAGE_TO_SLANDER = "https://thispersondoesnotexist.com/"

SLANDEROUS_SEED_HEADLINES = [
  f"{NAME_TO_SLANDER} convicted of stealing puppies",
  f"{NAME_TO_SLANDER} caught lying about growing the world's largest watermelon",
  f"{NAME_TO_SLANDER} accused of stealing priceless artifacts from Egypt",
  f"{NAME_TO_SLANDER} forged priceless works of modern art for decades",
  f"{NAME_TO_SLANDER} claimed to be Pokemon master, but caught in a lie",
  f"{NAME_TO_SLANDER} bought fake twitter followers to pretend to be a celebrity",
  f"{NAME_TO_SLANDER} caught in the act robbing a pet store",
  f"{NAME_TO_SLANDER} revealed as a foriegn spy for the undersea city of Atlantis",
  f"{NAME_TO_SLANDER} involved in blackmail scandal with King Trident of Atlantis",
  f"{NAME_TO_SLANDER} hid past crimes to get elected as Mayor of Otter Town",
  f"{NAME_TO_SLANDER} lied on tax returns to cover up past life as a Ninja Turtle",
  f"{NAME_TO_SLANDER} stole billions from investors in a new pet store",
  f"{NAME_TO_SLANDER} claims to be a Ninja Turtle but was actually lying",
  f"{NAME_TO_SLANDER} likely to be sentenced to 20 years in jail for chasing a cat into a tree",
  f"{NAME_TO_SLANDER} recieves record prison sentence for offensive smell",
  f"{NAME_TO_SLANDER} commits a multitude of crimes against dinosaurs",
]

# Which news website to 'clone'
#DOMAIN_STYLE_TO_COPY = "www.nytimes.com"
#RSS_FEEDS_OF_REAL_STORIES_TO_EMULATE = [
#  "https://rss.nytimes.com/services/xml/rss/nyt/US.xml",
#]
#http://feeds.bbci.co.uk/news/business/rss.xml
DOMAIN_STYLE_TO_COPY = "www.bbc.co.uk"
RSS_FEEDS_OF_REAL_STORIES_TO_EMULATE = [
  "http://feeds.bbci.co.uk/news/business/rss.xml",
]


### Step 2: Download Grover code and install requirements

In [6]:
%cd /content
!git clone https://github.com/rowanz/grover.git
%cd /content/grover
!python3 -m pip install regex jsonlines twitter-text-python feedparser

/content
Cloning into 'grover'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 90 (delta 3), reused 0 (delta 0), pack-reused 84
Unpacking objects: 100% (90/90), done.
/content/grover
     |████████████████████████████████| 194kB 4.0MB/s 
  Created wheel for twitter-text-python: filename=twitter_text_python-1.1.1-cp36-none-any.whl size=10043 sha256=4244a79dd8ef284837566529e55d522c6450a6328a3916d7e3bc815f4be21c35
  Stored in directory: /root/.cache/pip/wheels/08/a4/8b/fc095442f760d0103f128b052ca90c46485077541c5a6a86bc
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=8108d618f705dd9ffbe57b61aadd0e1595c10603b8a123835cfb2be6059b772e
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built twitter-text-python feedparser


### Step 3: Download Grover Pre-Trained 'Mega' Model

In [7]:
import os
import requests

model_type = "mega"

model_dir = os.path.join('/content/grover/models', model_type)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

for ext in ['data-00000-of-00001', 'index', 'meta']:
    r = requests.get(f'https://storage.googleapis.com/grover-models/{model_type}/model.ckpt.{ext}', stream=True)
    with open(os.path.join(model_dir, f'model.ckpt.{ext}'), 'wb') as f:
        file_size = int(r.headers["content-length"])
        if file_size < 1000:
            raise ValueError("File doesn't exist? idk")
        chunk_size = 1000
        for chunk in r.iter_content(chunk_size=chunk_size):
            f.write(chunk)
    print(f"Just downloaded {model_type}/model.ckpt.{ext}!", flush=True)

Just downloaded mega/model.ckpt.data-00000-of-00001!
Just downloaded mega/model.ckpt.index!
Just downloaded mega/model.ckpt.meta!


### Step 4: Generate Fake Blog Entries and Post to Wordpress

In [9]:
import tensorflow as tf
import numpy as np
import sys
import feedparser
import time
from datetime import datetime, timedelta
import requests
import base64
from ttp import ttp

sys.path.append('../')
from lm.modeling import GroverConfig, sample
from sample.encoder import get_encoder, _tokenize_article_pieces, extract_generated_target
import random


def get_fake_articles(domain):
    """
    Create article objects for each fake headline we have in 
    SLANDEROUS_SEED_HEADLINES suitable for feeding into Grover
    to generate the story body. The domain name is used to control
    the style of the text generated by Grover - i.e. bbc.co.uk would generate
    results in British English while nytimes.com would generate US English.
    """
    articles = []

    headlines_to_inject = SLANDEROUS_SEED_HEADLINES

    for fake_headline in headlines_to_inject:
        days_ago = random.randint(1, 7)
        pub_datetime = datetime.now() - timedelta(days=days_ago)

        publish_date = pub_datetime.strftime('%m-%d-%Y')
        iso_date = pub_datetime.isoformat()

        articles.append({
            'summary': "",
            'title': fake_headline,
            'text': '',
            'authors': ["Staff Writer"],
            'publish_date': publish_date,
            'iso_date': iso_date,
            'domain': domain,
            'image_url': IMAGE_TO_SLANDER,
            'tags': ['Breaking News', 'Investigations', 'Criminal Profiles'],
        })

    return articles


def get_articles_from_real_blog(domain, feed_url):
    """
    Given an RSS feed url, grab all the stories and format them as article objects
    suitable for feeding into Grover to generate replica stories.
    """
    feed_data = feedparser.parse(feed_url)
    articles = []
    for post in feed_data.entries:
        if 'published_parsed' in post:
            publish_date = time.strftime('%m-%d-%Y', post.published_parsed)
            iso_date = datetime(*post.published_parsed[:6]).isoformat()
        else:
            publish_date = time.strftime('%m-%d-%Y')
            iso_date = datetime.now().isoformat()

        if 'summary' in post:
            summary = post.summary
        else:
            summary = None

        tags = []
        if 'tags' in post:
            tags = [tag['term'] for tag in post['tags']]
            if summary is None:
                summary = ", ".join(tags)

        image_url = None
        if 'media_content' in post:
            images = post.media_content
            if len(images) > 0 and 'url' in images[0]:
                image_url = images[0]['url']
                # Hack for NYT images to fix tiny images in the RSS feed
                if "-moth" in image_url:
                    image_url = image_url.replace("-moth", "-threeByTwoMediumAt2X")

        if 'authors' in post:
            authors = list(map(lambda x: x["name"], post.authors))
        else:
            authors = ["Staff Writer"]

        articles.append({
            'summary': summary,
            'title': post.title,
            'text': '',
            'authors': authors,
            'publish_date': publish_date,
            'iso_date': iso_date,
            'domain': domain,
            'image_url': image_url,
            'tags': tags,
        })

    return articles





def format_generated_body_text_as_html(article_text, image_url=None):
    """
    Given the text of the news story, format it in html so it looks
    more realistic - add paragraph breaks, turn urls into links, etc.
    """
    # Add html links to twitter @ handles, hashtags and regular urls
    p = ttp.Parser()
    result = p.parse(article_text)
    article_text = result.html

    # Split the generated body into lines
    lines = article_text.split("\n")

    # Bold any short lines that look like section titles
    new_lines = []
    for line in lines:
        if len(line) < 80 and not "." in line:
            line = f"<b>{line}</b>"
        new_lines.append(line)

    # Add paragraph tags between lines
    article_text = "<p>".join(new_lines)
    
    # If we have an image for the story, put it at the top.
    if image_url is not None:
        article_text = f"<img src='{image_url}'><p>{article_text}"
    if image_url is None:
        article_text = f"<img src='https://thispersondoesnotexist.com/image.jpg'><p>{article_text}"
 
    return article_text


def generate_article_attribute(sess, encoder, tokens, probs, article, target='article'):

    """
    Given attributes about an article (title, author, etc), use that context to generate
    a replacement for one of those attributes using the Grover model.

    This function is based on the Grover examples distributed with the Grover code.
    """

    # Tokenize the raw article text
    article_pieces = _tokenize_article_pieces(encoder, article)

    # Grab the article elements the model careas about - domain, date, title, etc.
    context_formatted = []
    for key in ['domain', 'date', 'authors', 'title', 'article']:
        if key != target:
            context_formatted.extend(article_pieces.pop(key, []))

    # Start formatting the tokens in the way the model expects them, starting with
    # which article attribute we want to generate.
    context_formatted.append(encoder.__dict__['begin_{}'.format(target)])
    # Tell the model which special tokens (such as the end token) aren't part of the text
    ignore_ids_np = np.array(encoder.special_tokens_onehot)
    ignore_ids_np[encoder.__dict__['end_{}'.format(target)]] = 0

    # We are only going to generate one article attribute with a fixed
    # top_ps cut-off of 95%. This simple example isn't processing in batches.
    gens = []
    article['top_ps'] = [0.95]

    # Run the input through the TensorFlow model and grab the generated output
    tokens_out, probs_out = sess.run(
        [tokens, probs],
        feed_dict={
            # Pass real values for the inputs that the
            # model needs to be able to run.
            initial_context: [context_formatted],
            eos_token: encoder.__dict__['end_{}'.format(target)],
            ignore_ids: ignore_ids_np,
            p_for_topp: np.array([0.95]),
        }
    )

    # The model is done! Grab the results it generated and format the results into normal text.
    for t_i, p_i in zip(tokens_out, probs_out):
        extraction = extract_generated_target(output_tokens=t_i, encoder=encoder, target=target)
        gens.append(extraction['extraction'])

    # Return the generated text.
    return gens[-1]


# Ready to start grabbing RSS feeds
domain = DOMAIN_STYLE_TO_COPY
feed_urls = RSS_FEEDS_OF_REAL_STORIES_TO_EMULATE
articles = []

# Get the read headlines to look more realistic
for feed_url in feed_urls:
    articles += get_articles_from_real_blog(domain, feed_url)

# Toss in the slanderous articles
articles += get_fake_articles(domain)

# Randomize the order the articles are generated
random.shuffle(articles)

# Load the pre-trained "huge" Grover model with 1.5 billion params
model_config_fn = '/content/grover/lm/configs/mega.json'
model_ckpt = '/content/grover/models/mega/model.ckpt'
encoder = get_encoder()
news_config = GroverConfig.from_json_file(model_config_fn)

# Set up TensorFlow session to make predictions
tf_config = tf.ConfigProto(allow_soft_placement=True)

with tf.Session(config=tf_config, graph=tf.Graph()) as sess:
    # Create the placehodler TensorFlow input variables needed to feed data to Grover model
    # to make new predictions.
    initial_context = tf.placeholder(tf.int32, [1, None])
    p_for_topp = tf.placeholder(tf.float32, [1])
    eos_token = tf.placeholder(tf.int32, [])
    ignore_ids = tf.placeholder(tf.bool, [news_config.vocab_size])

    # Load the model config to get it set up to match the pre-trained model weights
    tokens, probs = sample(
        news_config=news_config,
        initial_context=initial_context,
        eos_token=eos_token,
        ignore_ids=ignore_ids,
        p_for_topp=p_for_topp,
        do_topk=False
    )

    # Restore the pre-trained Grover 'huge' model weights
    saver = tf.train.Saver()
    saver.restore(sess, model_ckpt)

    # START MAKING SOME FAKE NEWS!!
    # Loop through each headline we scraped from an RSS feed or made up
    for article in articles:
        print(f"Building article from headline '{article['title']}'")

        # If the headline is one we made up about a specific person, it needs special handling
        if NAME_TO_SLANDER in article['title']:
            # The first generated article may go off on a tangent and not include the target name.
            # In that case, re-generate the article until it at least talks about our target person
            attempts = 0
            while NAME_TO_SLANDER not in article['text']:

                # Generate article body given the context of the real blog title
                article['text'] = generate_article_attribute(sess, encoder, tokens, probs, article, target="article")

                # If the Grover model never manages to generate a good article about the target victim,
                # give up after 10 tries so we don't get stuck in an infinite loop
                attempts += 1
                print('try to attemps once!')
                print(attempts)
                if attempts > 3:
                  print('give up faking this article!')
                  break
        # If the headline was scraped from an RSS feed, we can just blindly generate an article
        else:
            article['text'] = generate_article_attribute(sess, encoder, tokens, probs, article, target="article")

        # Now, generate a fake headline that better fits the generated article body
        # This replaces the real headline so none of the original article content remains
        article['title'] = generate_article_attribute(sess, encoder, tokens, probs, article, target="title")

        # Grab generated text results so we can post them to WordPress
        article_title = article['title']
        article_text = article['text']
        article_date = article["iso_date"]
        article_image_url = article["image_url"]
        article_tags = article['tags']

        # Make the article body look more realistic - add spacing, link Twitter handles and hashtags, etc.
        # You could add more advanced pre-processing here if you wanted.
        article_text = format_generated_body_text_as_html(article_text, article_image_url)

        print(f" - Generated fake article titled '{article_title}'")
        # Post result to target Wordpress blog
        #post_to_wordpress_blog(article_title, article_text, article_tags, article_date)
        #post and activate new post
        post = WordPressPost()
        post.title = article_title
        post.content = article_text
        post.post_status = 'publish'
        post.terms_names = {
          'post_tag': article_tags,
          'category': ['Thisisfake', 'fakenews']
        }
        wp.call(NewPost(post))

INFO:tensorflow:Restoring parameters from /content/grover/models/mega/model.ckpt
Building article from headline 'HSBC customers hit by two IT glitches within hours'
 - Generated fake article titled 'HSBC reports major banking problems as customers struggle to get transactions processed'
Building article from headline 'Thomas Cook: 'They told me the money was safe''
 - Generated fake article titled 'The pitfalls of holiday money: Post-Brexit uncertainty costs you'
Building article from headline 'How the Christmas cracker became a million dollar idea'
 - Generated fake article titled 'What a Salvation Army Red Kettle Could Look Like'
Building article from headline 'Boeing 'is not a trustworthy company anymore''
 - Generated fake article titled 'Boeing’s Bob Moritz invokes memory of plane’s grim fate and slams CEO’s defense of new plane quality'
Building article from headline 'Airbnb is not an estate agent, EU court rules'
 - Generated fake article titled 'Here’s what the ECJ has to say a

KeyboardInterrupt: ignored

**Get more RAM:**

In [1]:
!pip install -q tqdm
import tqdm
from google.colab import auth
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload


def download_file_from_google_drive(file_id = None, out_file_name = None):
  assert file_id is not None and out_file_name is not None
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  pbar = tqdm.tqdm(total=100, desc=out_file_name)

  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    status, done = downloader.next_chunk()
    pbar.update(status.progress() * 100)
    #print("Downloaded: ", int(status.progress() * 100))

  downloaded.seek(0)
  #print('Downloaded file contents are: {}'.format(downloaded.read()[:10]))
  with open(out_file_name, 'wb') as out:
    out.write(downloaded.read())
  print("Data downloaded to: ", out_file_name)
  return out_file_name
 

def save_file_to_google_drive(local_filename, dest_filename, mimetype = 'application/octet-stream'):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')

  file_metadata = {
    'name': dest_filename,
    'mimeType': mimetype
  }
  media = MediaFileUpload(local_filename, 
                          mimetype=mimetype,
                          resumable=True)
  created = drive_service.files().create(body=file_metadata,
                                         media_body=media,
                                         fields='id').execute()
  print('File ID: {}'.format(created.get('id')))
  return created.get('id')
download_file_from_google_drive('1L-xyHzHNsQhPGFXdi2WbjZ2_lE4BPmzF','123.zip')

123.zip: 3742.674538929257it [03:14, 48.56it/s]

Data downloaded to:  123.zip


'123.zip'

In [2]:
!unzip 123.zip

Archive:  123.zip
   creating: content/Mask_RCNN/
  inflating: content/Mask_RCNN/setup.cfg  
  inflating: content/Mask_RCNN/MANIFEST.in  
   creating: content/Mask_RCNN/dist/
  inflating: content/Mask_RCNN/dist/mask_rcnn-2.1-py3.6.egg  
   creating: content/Mask_RCNN/coco_model/
   creating: content/Mask_RCNN/coco_model/hat_dataset/
   creating: content/Mask_RCNN/coco_model/hat_dataset/train/
  inflating: content/Mask_RCNN/coco_model/hat_dataset/train/46617494_004_c.jpeg  
  inflating: content/Mask_RCNN/coco_model/hat_dataset/train/42281832_001_a.jpeg  
  inflating: content/Mask_RCNN/coco_model/hat_dataset/train/Free-People-Harlow-Cable-Knit-Beanie-Cherry-Wine-2.jpg  
  inflating: content/Mask_RCNN/coco_model/hat_dataset/train/th.jpeg  
  inflating: content/Mask_RCNN/coco_model/hat_dataset/train/41096801_066_a.jpeg  
  inflating: content/Mask_RCNN/coco_model/hat_dataset/train/funny-hat-cap-pictures-e41b564ab5c61a693cf915dfa4d7b201-silly-hats-funny-hats.jpg  
  inflating: content/Mask_R

In [0]:
print(article_image_url)

None
